### Problem statement
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable.

Metric Submissions are evaluated on Mean-Squared-Error (MSE). Submission File Format The file should contain a header and have the following format:

Id,SalePrice
1461,169000.1
1462,187724.1233

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

### Read Dataset

In [2]:
import pandas as pd
A = pd.read_csv("C:/Users/kruta/Desktop/DATA_SCI-ANA/Sanyuta/June/11-06-2022/training_set.csv")
B = pd.read_csv("C:/Users/kruta/Desktop/DATA_SCI-ANA/Sanyuta/June/11-06-2022/testing_set.csv")

### Replace Items

In [3]:
A.Alley = A.Alley.fillna("No alley access")
A.BsmtQual = A.BsmtQual.fillna("No Basement")
A.BsmtCond = A.BsmtCond.fillna("No Basement")
A.BsmtExposure = A.BsmtExposure.fillna("No Basement")
A.BsmtFinType1 = A.BsmtFinType1.fillna("No Basement")
A.BsmtFinType2 = A.BsmtFinType2.fillna("No Basement")
A.FireplaceQu = A.FireplaceQu.fillna("No Fireplace")
A.GarageType = A.GarageType.fillna("No Garage")
A.GarageFinish = A.GarageFinish.fillna("No Garage")
A.GarageQual = A.GarageQual.fillna("No Garage")
A.GarageCond = A.GarageCond.fillna("No Garage")
A.PoolQC = A.PoolQC.fillna("No Pool")
A.Fence = A.Fence.fillna("No Fence")
A.MiscFeature = A.MiscFeature.fillna("None")

In [4]:
B.Alley = B.Alley.fillna("No alley access")
B.BsmtQual = B.BsmtQual.fillna("No Basement")
B.BsmtCond = B.BsmtCond.fillna("No Basement")
B.BsmtExposure = B.BsmtExposure.fillna("No Basement")
B.BsmtFinType1 = B.BsmtFinType1.fillna("No Basement")
B.BsmtFinType2 = B.BsmtFinType2.fillna("No Basement")
B.FireplaceQu = B.FireplaceQu.fillna("No Fireplace")
B.GarageType = B.GarageType.fillna("No Garage")
B.GarageFinish = B.GarageFinish.fillna("No Garage")
B.GarageQual = B.GarageQual.fillna("No Garage")
B.GarageCond = B.GarageCond.fillna("No Garage")
B.PoolQC = B.PoolQC.fillna("No Pool")
B.Fence = B.Fence.fillna("No Fence")
B.MiscFeature = B.MiscFeature.fillna("None")

In [5]:
for i in A.columns:
    if(A[i].dtypes == "object"):
        x = A[i].mode()
        A[i]=A[i].fillna(x)
    else:
        x = A[i].mean()
        A[i]=A[i].fillna(x)

In [6]:
for i in B.columns:
    if(B[i].dtypes == "object"):
        x = B[i].mode()[0]
        B[i]=A[i].fillna(x)
    else:
        x = B[i].mean()
        B[i]=B[i].fillna(x)

### Checking correlation

In [7]:
A.corr()[["SalePrice"]]

,SalePrice
Id,-0.021917
MSSubClass,-0.084284
LotFrontage,0.334901
LotArea,0.263843
OverallQual,0.790982
OverallCond,-0.077856
YearBuilt,0.522897
YearRemodAdd,0.507101
MasVnrArea,0.475241
BsmtFinSF1,0.386420


In [8]:
X = A.drop(labels = ["Id","SalePrice"],axis = 1)
Y = A[["SalePrice"]]


In [9]:
X = X.drop(labels = ["MSSubClass","OverallCond","BsmtFinSF2","LowQualFinSF","BsmtHalfBath","LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFullBath","HalfBath","BedroomAbvGr","KitchenAbvGr","Fireplaces","GarageYrBlt","WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal","MoSold","YrSold"],axis = 1 )
#drop columns based on correlation

### Preprocessing the data 

In [10]:
cat = []
con = []

for i in X.columns:
    if (X[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [11]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X1 = pd.DataFrame(ss.fit_transform(A[con]),columns=con)

### Remove the outliers 

In [12]:
outliers = []
for i in X1.columns:
    outliers.extend(list(X1[(X1[i]>3) | (X1[i]<-3)].index))

In [13]:
import numpy as np
outliers = np.unique(outliers)

In [14]:
X1 = X1.drop(index=outliers,axis=0)

### Checking correlation of categorical columns 

In [15]:
for i in cat:
    from statsmodels.formula.api import ols
    rel = "SalePrice ~" + str(i)
    model = ols(rel,A).fit()
    from statsmodels.stats.anova import anova_lm
    anova_results = anova_lm(model)
    print(rel, "--> ",round(anova_results.iloc[0,4],4))

SalePrice ~MSZoning -->  0.0
SalePrice ~Street -->  0.117
SalePrice ~Alley -->  0.0
SalePrice ~LotShape -->  0.0
SalePrice ~LandContour -->  0.0
SalePrice ~Utilities -->  0.5847
SalePrice ~LotConfig -->  0.0
SalePrice ~LandSlope -->  0.1414
SalePrice ~Neighborhood -->  0.0
SalePrice ~Condition1 -->  0.0
SalePrice ~Condition2 -->  0.0434
SalePrice ~BldgType -->  0.0
SalePrice ~HouseStyle -->  0.0
SalePrice ~RoofStyle -->  0.0
SalePrice ~RoofMatl -->  0.0
SalePrice ~Exterior1st -->  0.0
SalePrice ~Exterior2nd -->  0.0
SalePrice ~MasVnrType -->  0.0
SalePrice ~ExterQual -->  0.0
SalePrice ~ExterCond -->  0.0
SalePrice ~Foundation -->  0.0
SalePrice ~BsmtQual -->  0.0
SalePrice ~BsmtCond -->  0.0
SalePrice ~BsmtExposure -->  0.0
SalePrice ~BsmtFinType1 -->  0.0
SalePrice ~BsmtFinType2 -->  0.0
SalePrice ~Heating -->  0.0008
SalePrice ~HeatingQC -->  0.0
SalePrice ~CentralAir -->  0.0
SalePrice ~Electrical -->  0.0
SalePrice ~KitchenQual -->  0.0
SalePrice ~Functional -->  0.0005
SalePrice 

In [16]:
cat.remove("Street")
cat.remove("Utilities")
cat.remove("LandSlope")

In [17]:
X2 = pd.get_dummies(X[cat])

In [18]:
Xnew = X1.join(X2)

In [19]:
Y = Y.drop(index=outliers,axis=0)

In [20]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=31)

### OLS Regression Results

In [21]:
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     61.20
Date:                Tue, 28 Jun 2022   Prob (F-statistic):               0.00
Time:                        23:48:19   Log-Likelihood:                -12606.
No. Observations:                1126   AIC:                         2.564e+04
Df Residuals:                     914   BIC:                         2.670e+04
Df Model:                         211                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     2.306e+04   2754.605      8.370      0.000    1.76e+04    2.85e+04
OverallQual               1.089e+04   1396.400      7.797      0.000    8146.556    1.36e+04
YearBuilt                 3782.3779   2201.887      1.718      0.086    -538.963    8103.719
YearRemodAdd              3941.8787   1019.385      3.867      0.000    1941.271    5942.487
BsmtUnfSF                -9729.8809   1236.193     -7.871      0.000   -1.22e+04   -7303.775
TotalBsmtSF               1.667e+04   2219.729      7.511      0.000    1.23e+04     2.1e+04
1stFlrSF                   107.0746   8136.934      0.013      0.990   -1.59e+04    1.61e+04
2ndFlrSF                  5675.8667   8606.706      0.659      0.510   -1.12e+04    2.26e+04
GrLivArea                 2.573e+04   1.09e+04      2.357      0.019    4308.755    4.72e+04
FullBath                   344.1238   1083.446      0.318      0.751   -1782.207    2470.454
TotRmsAbvGrd             -1007.4178   1422.801     -0.708      0.479   -3799.755    1784.919
GarageCars                3540.2279   1705.155      2.076      0.038     193.754    6886.702
GarageArea                3243.5681   1657.017      1.957      0.051      -8.431    6495.567
MSZoning_C (all)         -1.888e+04   8896.955     -2.122      0.034   -3.63e+04   -1416.069
MSZoning_FV               1.823e+04   6004.809      3.037      0.002    6449.853       3e+04
MSZoning_RH               6420.9436   6133.249      1.047      0.295   -5615.943    1.85e+04
MSZoning_RL               1.224e+04   3209.447      3.815      0.000    5944.595    1.85e+04
MSZoning_RM               5033.0024   3592.683      1.401      0.162   -2017.865    1.21e+04
Alley_Grvl                9144.2691   3185.616      2.870      0.004    2892.298    1.54e+04
Alley_No alley access     7407.1407   2213.800      3.346      0.001    3062.420    1.18e+04
Alley_Pave                6503.6391   3408.628      1.908      0.057    -186.007    1.32e+04
LotShape_IR1             -2166.4839   2556.200     -0.848      0.397   -7183.188    2850.220
LotShape_IR2              8492.6458   3654.044      2.324      0.020    1321.356    1.57e+04
LotShape_IR3              1.781e+04   6504.866      2.738      0.006    5041.792    3.06e+04
LotShape_Reg             -1079.1128   2619.219     -0.412      0.680   -6219.495    4061.270
LandContour_Bnk           6225.0909   3134.901      1.986      0.047      72.651    1.24e+04
LandContour_HLS           1.057e+04   3225.187      3.277      0.001    4240.660    1.69e+04
LandContour_Low           1950.6806   3926.916      0.497      0.619   -5756.139    9657.501
LandContour_Lvl           4308.9848   2114.758      2.038      0.042     158.640    8459.330
LotConfig_Corner          9270.7055   3066.413      3.023      0.003    3252.678    1.53e+04
LotConfig_CulD

In [22]:
cols_to_drop = model.pvalues.sort_values().index[-1]

In [23]:
cols_to_drop

'MiscFeature_TenC'

In [24]:
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9186

In [25]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9186

In [26]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9186

In [27]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9186

In [28]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9186

In [29]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9186

In [30]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9187

In [31]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9188

In [32]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9188

In [33]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9188

In [34]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9188

In [35]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9188

In [36]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9188

In [37]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9188

In [38]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9189

In [39]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.919

In [40]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.919

In [41]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.919

In [42]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.919

In [43]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.919

In [44]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9191

In [45]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9191

In [46]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91908

In [47]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.9192

In [48]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91917

In [49]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91926

In [50]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91934

In [51]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91934

In [52]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91943

In [53]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91951

In [54]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91951

In [55]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91951

In [56]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9196

In [57]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91968

In [58]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91968

In [59]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91977

In [60]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91985

In [61]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91993

In [62]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91993

In [63]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91993

In [64]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.91993

In [65]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)


0.92002

In [66]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9201

In [67]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9201

In [68]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)


0.9201

In [69]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9201

In [70]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9201

In [71]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92018

In [72]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92026

In [73]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92034

In [74]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92042

In [75]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92042

In [76]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9205

In [77]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9205

In [78]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9205

In [79]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92058

In [80]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92058

In [81]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92058

In [82]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92066

In [83]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92066

In [84]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92073

In [85]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92081

In [86]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92088

In [87]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92088

In [88]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92088

In [89]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92088

In [90]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92097

In [91]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92104

In [92]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92111

In [93]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92118

In [94]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92126

In [95]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92133

In [96]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9214

In [97]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92147

In [98]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92154

In [99]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92162

In [100]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92169

In [101]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92169

In [102]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92169

In [103]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92176

In [104]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92182

In [105]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92189

In [106]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92189

In [107]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92189

In [108]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92189

In [109]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92195

In [110]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92202

In [111]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92209

In [112]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92209

In [113]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92216

In [114]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92216

In [115]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92224

In [116]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9223

In [117]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92236

In [118]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92243

In [119]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92249

In [120]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92255

In [121]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92261

In [122]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92267

In [123]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92273

In [124]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92278

In [125]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92285

In [126]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9229

In [127]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92296

In [128]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92304

In [129]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92311

In [130]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92316

In [131]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92321

In [132]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92326

In [133]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92331

In [134]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92336

In [135]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92341

In [136]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92346

In [137]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92351

In [138]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92356

In [139]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9236

In [140]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92365

In [141]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92369

In [142]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92375

In [143]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92379

In [144]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92383

In [145]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92387

In [146]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9239

In [147]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92393

In [148]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92397

In [149]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92399

In [150]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92402

In [151]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92404

In [152]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92406

In [153]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92407

In [154]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92409

In [155]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92409

In [156]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92417

In [157]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92418

In [158]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92423

In [159]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92424

In [160]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92428

In [161]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92429

In [162]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92435

In [163]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92435

In [164]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92436

In [165]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92436

In [166]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92443

In [167]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.9245

In [168]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92453

In [169]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92455

In [170]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92456

In [171]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92455

In [172]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92455

In [173]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92454

In [174]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.92453

### Check for Overfitting

In [175]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(xtrain,ytrain)
tr_pred = model.predict(xtrain)
ts_pred = model.predict(xtest)

from sklearn.metrics import mean_squared_error
tr_err = round(mean_squared_error(ytrain,tr_pred),2)
ts_err = round(mean_squared_error(ytest,ts_pred),2)

In [176]:
tr_err

319724999.93

In [177]:
ts_err

8.340232743657757e+28

### Lasso and Ridge to reduce overfitting

In [178]:
from sklearn.linear_model import Ridge,Lasso
rr = Ridge()
ls = Lasso()

In [179]:
q = 0.01
w = []
for i in range(0,300):
    w.append(q)
    q = q + 0.01

### GridSearchCV for alpha value

In [180]:
from sklearn.model_selection import GridSearchCV

tuning_grid = {"alpha":w}
cv = GridSearchCV(rr,tuning_grid,scoring="neg_mean_squared_error",cv=4)
cvmodel = cv.fit(Xnew,Y)
print(cvmodel.best_params_ ,"alpha value using Ridge")

{'alpha': 1.9500000000000015} alpha value using Ridge


In [181]:
from sklearn.model_selection import GridSearchCV

tuning_grid = {"alpha":w}
cv = GridSearchCV(ls,tuning_grid,scoring="neg_mean_squared_error",cv=4)
cvmodel = cv.fit(Xnew,Y)
print(cvmodel.best_params_ ,"alpha value using Lasso")

{'alpha': 2.99999999999998} alpha value using Lasso


In [182]:
rr = Ridge(alpha=1.95)
model = rr.fit(xtrain,ytrain)
tr_pred = model.predict(xtrain) 
ts_pred = model.predict(xtest)
tr_err = round(mean_squared_error(ytrain,tr_pred),2)
ts_err = round(mean_squared_error(ytest,ts_pred),2)
print(tr_err,ts_err)

330383888.09 429972174.88


In [183]:
    ls = Lasso(alpha= 3)
    model = rr.fit(xtrain,ytrain)
    tr_pred = model.predict(xtrain) 
    ts_pred = model.predict(xtest)
    tr_err = round(mean_squared_error(ytrain,tr_pred),2)
    ts_err = round(mean_squared_error(ytest,ts_pred),2)
    print(tr_err,ts_err)

330383888.09 429972174.88


### Making test set ready for prediction

In [184]:
cols_to_keep = xtrain.columns

### Preprocessing

In [185]:
cat = []
con = []
for i in B.columns:
    if(B[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [186]:
Xtest1 = pd.DataFrame(ss.fit_transform(B[con]),columns = con)

In [187]:
Xtest2 = pd.get_dummies(B[cat])

In [188]:
Xtest_new = Xtest1.join(Xtest2)

In [189]:
final_Xtest = Xtest_new[cols_to_keep]

In [190]:
pred = model.predict(final_Xtest)

In [191]:
T = B[["Id"]]
T[["SalePrice"]]=pred

In [192]:
T.head()

,Id,SalePrice
0,1461,126953.573289
1,1462,197609.309471
2,1463,198064.373000
3,1464,201712.863948
4,1465,215494.617469


### Exporting results as csv

In [193]:
T.to_csv("C:/Users/kruta/Desktop/submission.csv")